<a href="https://colab.research.google.com/github/srirampattabiraman/Extensive-NLP-and-Deep-Learning/blob/main/session_5/Sentiment_Analysis_using_Augumentated_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#We are mounting drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from tqdm import tqdm
import random
import sys
import pandas as pd
import numpy as np
import os
import random
import torch, torchtext
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.legacy import data
from torchtext.legacy.datasets import IMDB
from torchtext.legacy.data import Field, LabelField, BucketIterator,Example

In [37]:
##Loading Augumented data
augumented_data = pd.read_csv("drive/MyDrive/END_DATA/session4/stanfordSentimentTreebank/transformed_data.csv", index_col=[0])

In [38]:
augumented_data.drop_duplicates(inplace=True)
augumented_data.reset_index(inplace=True,drop=True)

In [40]:
print(augumented_data.head())
print("shape of augumented_data dataframe is {}".format(augumented_data.shape))
augumented_data.columns = ['sentence','sentiment']

                                            sentence    label
0  The Rock is destined to be the new `` Conan ''...  0.69444
1  Rock is 21st '' that going to make a even than...  0.69444
2  Century Rock the destined to be '' 21st The 's...  0.69444
3  The Rock is destined to be the new `` Conan ''...  0.69444
4  destined 21st 's `` Conan and that he 's to gr...  0.69444
shape of augumented_data dataframe is (17115, 2)


In [41]:
train_data = pd.read_csv("drive/MyDrive/END_DATA/session4/stanfordSentimentTreebank/train.csv")
test_data = pd.read_csv("drive/MyDrive/END_DATA/session4/stanfordSentimentTreebank/test.csv")
dev_data = pd.read_csv("drive/MyDrive/END_DATA/session4/stanfordSentimentTreebank/dev.csv")

In [42]:
Train_df = pd.concat([train_data,test_data])
Test_df = dev_data.copy()

In [43]:
Train_df.head(2)


,sentence,id,sentiment,fine,coarse
0,The Rock is destined to be the 21st Century 's...,226166.0,0.69444,positive,positive
1,The gorgeously elaborate continuation of `` Th...,226300.0,0.83333,very positive,positive


In [44]:
Train_df = pd.concat([Train_df,augumented_data])
Train_df.reset_index(inplace=True,drop = True)

In [45]:
Train_df.head(2)

,sentence,id,sentiment,fine,coarse
0,The Rock is destined to be the 21st Century 's...,226166.0,0.69444,positive,positive
1,The gorgeously elaborate continuation of `` Th...,226300.0,0.83333,very positive,positive


In [46]:
print(Train_df.shape)
print(Test_df.shape)

(27869, 5)
(1101, 5)


In [47]:
print("The Total null values in Train Data:- ",Train_df['sentiment'].isnull().sum())
print("The Total null values in Test Data:- ",Test_df['sentiment'].isnull().sum())

The Total null values in Train Data:-  512
The Total null values in Test Data:-  57


In [49]:
Train_df.sentiment.interpolate(method ='linear', limit_direction ='forward',inplace=True)
Test_df.sentiment.interpolate(method ='linear', limit_direction ='forward',inplace=True)

In [51]:
def discretize_label(label):
    if label <= 0.05*100: return 'Class1'
    if label <= 0.1*100: return 'Class2'
    if label <= 0.15*100: return 'Class3'
    if label <= 0.2*100: return 'Class4'
    return 'Class5'

In [52]:
Train_df['label'] = (Train_df['sentiment'] * 24).clip(0, 24).astype('int')
Test_df['label'] = (Test_df['sentiment'] * 24).clip(0, 24).astype('int')

Train_df['label'] = Train_df['label'].apply(discretize_label)
Test_df['label'] = Test_df['label'].apply(discretize_label)

In [53]:
Description = Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Sentiment = LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [54]:
fields = [('Description', Description),('Sentiments',Sentiment)]

Armed with our declared fields, lets convert from pandas to list to torchtext. We could also use TabularDataset to apply that definition to the CSV directly but showing an alternative approach too.

In [55]:
fields

[('Description', <torchtext.legacy.data.field.Field at 0x7f0ec6dd2150>),
 ('Sentiments', <torchtext.legacy.data.field.LabelField at 0x7f0ec7bb2f90>)]

In [56]:
Train_df.reset_index(inplace=True,drop = True)
Test_df.reset_index(inplace=True,drop = True)

In [58]:
train_example = [torchtext.legacy.data.Example.fromlist([Train_df.sentence[i],Train_df.label[i]], fields) for i in range(Train_df.shape[0])]
test_example = [torchtext.legacy.data.Example.fromlist([Test_df.sentence[i],Test_df.label[i]], fields) for i in range(Test_df.shape[0])]

train_data = torchtext.legacy.data.Dataset(train_example, fields,)
test_data = torchtext.legacy.data.Dataset(test_example, fields)

we are going to each element and converting each tweet and its labels, assigning fields object to it.

In [61]:
Description.build_vocab(train_data)
Sentiment.build_vocab(train_data)

In [62]:
print('Size of input vocab : ', len(Description.vocab))
print('Size of label vocab : ', len(Sentiment.vocab))
print('Top 10 words appreared repeatedly :', list(Description.vocab.freqs.most_common(10)))
print('Labels : ', Sentiment.vocab.stoi)

Size of input vocab :  19311
Size of label vocab :  5
Top 10 words appreared repeatedly : [('to', 29452), ("'s", 28819), ('`', 26603), ('.', 22937), (',', 21903), ('the', 20476), ('and', 18553), ('a', 18320), ('-', 16261), ('is', 15993)]
Labels :  defaultdict(None, {'Class4': 0, 'Class2': 1, 'Class1': 2, 'Class3': 3, 'Class5': 4})


In [65]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [122]:
BATCH_SIZE = 32

train_iterator,test_iterator = torchtext.legacy.data.BucketIterator.splits(
    (train_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.Description),
    sort_within_batch=True, # necessary for packed_padded_sequence
    device = device,shuffle=True)

In [68]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Description.vocab.stoi, tokens)

In [69]:
# Model class
import torch.nn.functional as F
class Model(nn.Module):
    def __init__(self, input_dim, output_dim,emb_dim, hidden_dim, n_layers, dropout):
        # input_dim <--- vocabulary size
        # output_dim <--- len ([positive, negative]) == 2 
        # emb_dim <--- embedding dimension of embedding matrix

        super(Model, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.encoder = nn.LSTM(emb_dim, hidden_dim, n_layers, dropout=dropout,batch_first=True)

        self.fc1 = nn.Linear(hidden_dim, 512)
        self.fc2 = nn.Linear(512, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src,Len):
        # shape: [source_len, batch_size]
        embedded = self.dropout(self.embedding(src)) # shape: [src_len, batch_size, embed_dim]
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, Len.to('cpu'),batch_first=True)
        output, (hidden, cell) = self.encoder(packed) 
        # output shape -> [batch, hidden_dim]
        # hiddden shape -> [n_layers, batch, hidden_dim]
        # cell shape -> [n_layers, batch, hidden_dim]
        output = F.relu(self.fc1(hidden))
#         output = self.dropout(output)
        output = self.fc2(output)
        output = F.softmax(output, dim=1)
#         output = F.relu(output)
        
        return output[-1].squeeze(0)

In [89]:
#initializing variables and hyper parameters
INPUT_DIM = len(Description.vocab)
OUTPUT_DIM = len(Sentiment.vocab)
print(OUTPUT_DIM)

EMBEDDING_DIM = 512
HIDDEN_DIM = 512

N_LAYERS = 5
DROPOUT = 0.2

# initializing our model
model = Model(INPUT_DIM, OUTPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, N_LAYERS, DROPOUT).to(device)

5


In [90]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

Model(
  (embedding): Embedding(19311, 512)
  (encoder): LSTM(512, 512, num_layers=5, batch_first=True, dropout=0.2)
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=5, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
The model has 20,658,693 trainable parameters


In [91]:
train_loss = []
train_accuracy = []
test_loss = []
test_accuracy = []
validation_loss = []
validation_accuracy = []

## Model Training and Evaluation

In [92]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [135]:
def train(EPOCH,model, iterator, optimizer=optimizer, criterion=criterion, clip=1,):
  # print("printing iterator val:", iterator)
  model.train()
  epoch_loss = 0
  total_correct = 0
  total_count = 0
  pbar = tqdm(iterator)
  # print("printing iterator val:", enumerate(iterator))
  # for x in enumerate(iterator):
  #   print(x)
  for i, batch in enumerate(pbar):
      
    src,data_len = batch.Description
    src = src.to(device)
    trg = batch.Sentiments.to(device)
    trg = trg.long()
        
    optimizer.zero_grad()
        
    output= model(src,data_len).squeeze()
    total_correct += torch.sum(torch.eq(output.argmax(1), trg))
    total_count+=len(trg)
    acc = binary_accuracy(output, trg)   
    loss = criterion(output, trg)
        
    loss.backward() 
    optimizer.step()
        
    epoch_loss += loss.item()
        
    pbar.set_description(desc= f"\033[1mEpoch {EPOCH}  Train data \033[0m Batch No : {i} Epoch Loss: {epoch_loss:.3f} Loss : {loss.item():.3f} Accuracy : {total_correct/total_count * 100 :.2f}% " )
    
    train_accuracy.append(acc)
    mean_loss = epoch_loss / len(iterator)
    train_loss.append(mean_loss)
    
    scheduler.step(mean_loss)

In [136]:
def evaluate(EPOCH, model, iterator, criterion,typ_loader):
  print("printing iterator--->", iterator)
  epoch_loss = 0
  epoch_acc = 0
  model.eval()
  pbar  = tqdm(iterator)
  with torch.no_grad():
    for i,batch in enumerate(iterator):
      # print(i,batch)
      src,data_len = batch.Description
      src = src.to(device)
      trg = batch.Sentiments.to(device)
      trg = trg.long()
      predictions = model(src,data_len).squeeze()
            
      loss = criterion(predictions, trg)
            
      acc = binary_accuracy(predictions, trg)

      epoch_loss += loss.item()
      epoch_acc += acc
      if typ_loader == 'Valid data':
        validation_loss.append(loss)
        validation_accuracy.append(acc)
      elif typ_loader == 'Test data':
        test_loss.append(loss)
        test_accuracy.append(acc)
      pbar.set_description(desc= f'\033[4mEpoch {EPOCH}  {typ_loader} Batch No : {i} Epoch Loss: {epoch_loss:.3f}  Loss : {loss.item():.3f} | Accuracy : {epoch_acc / len(iterator)* 100 :.2f}%\033[0m' )

In [ ]:
total_epoch = 10
for epoch in range(total_epoch):
    result = train(epoch, model, train_iterator)
    evaluate(epoch,model,test_iterator,criterion,'TEST DATA')

#     evaluate(epoch,model,valid_iterator,criterion,'Valid data')

Streaming output truncated to the last 5000 lines.

Epoch 0  Train data  Batch No : 615 Epoch Loss: 982.358 Loss : 1.579 Accuracy : 57.91% :  70%|███████   | 613/871 [00:27<00:11, 21.95it/s]













Epoch 0  Train data  Batch No : 615 Epoch Loss: 982.358 Loss : 1.579 Accuracy : 57.91% :  71%|███████   | 616/871 [00:27<00:11, 23.18it/s]













Epoch 0  Train data  Batch No : 616 Epoch Loss: 983.947 Loss : 1.589 Accuracy : 57.93% :  71%|███████   | 616/871 [00:27<00:11, 23.18it/s]













Epoch 0  Train data  Batch No : 617 Epoch Loss: 985.540 Loss : 1.593 Accuracy : 57.86% :  71%|███████   | 616/871 [00:27<00:11, 23.18it/s]













Epoch 0  Train data  Batch No : 618 Epoch Loss: 987.153 Loss : 1.613 Accuracy : 57.80% :  71%|███████   | 616/871 [00:27<00:11, 23.18it/s]













Epoch 0  Train data  Batch No : 618 Epoch Loss: 987.153 Loss : 1.613 Accuracy : 57.80% :  71%|███████   | 619/871 [00:27<00:10, 24.07it/s]













Epoch 0  Train data  Batch No : 619 

printing iterator---> <torchtext.legacy.data.iterator.BucketIterator object at 0x7f0e63a19d90>


Streaming output truncated to the last 5000 lines.

Epoch 1  Train data  Batch No : 615 Epoch Loss: 981.814 Loss : 1.620 Accuracy : 55.28% :  70%|███████   | 614/871 [00:27<00:10, 23.93it/s]













Epoch 1  Train data  Batch No : 616 Epoch Loss: 983.391 Loss : 1.578 Accuracy : 55.31% :  70%|███████   | 614/871 [00:27<00:10, 23.93it/s]













Epoch 1  Train data  Batch No : 616 Epoch Loss: 983.391 Loss : 1.578 Accuracy : 55.31% :  71%|███████   | 617/871 [00:27<00:10, 24.80it/s]













Epoch 1  Train data  Batch No : 617 Epoch Loss: 984.975 Loss : 1.584 Accuracy : 55.34% :  71%|███████   | 617/871 [00:27<00:10, 24.80it/s]













Epoch 1  Train data  Batch No : 618 Epoch Loss: 986.587 Loss : 1.611 Accuracy : 55.41% :  71%|███████   | 617/871 [00:27<00:10, 24.80it/s]













Epoch 1  Train data  Batch No : 619 Epoch Loss: 988.172 Loss : 1.586 Accuracy : 55.36% :  71%|███████   | 617/871 [00:27<00:10, 24.80it/s]













Epoch 1  Train data  Batch No : 619 

printing iterator---> <torchtext.legacy.data.iterator.BucketIterator object at 0x7f0e63a19d90>


Streaming output truncated to the last 5000 lines.
Epoch 2  Train data  Batch No : 613 Epoch Loss: 978.252 Loss : 1.583 Accuracy : 54.47% :  70%|███████   | 614/871 [00:27<00:10, 23.42it/s]













Epoch 2  Train data  Batch No : 614 Epoch Loss: 979.867 Loss : 1.615 Accuracy : 54.42% :  70%|███████   | 614/871 [00:27<00:10, 23.42it/s]













Epoch 2  Train data  Batch No : 615 Epoch Loss: 981.447 Loss : 1.580 Accuracy : 54.49% :  70%|███████   | 614/871 [00:27<00:10, 23.42it/s]













Epoch 2  Train data  Batch No : 616 Epoch Loss: 983.038 Loss : 1.591 Accuracy : 54.52% :  70%|███████   | 614/871 [00:27<00:10, 23.42it/s]













Epoch 2  Train data  Batch No : 616 Epoch Loss: 983.038 Loss : 1.591 Accuracy : 54.52% :  71%|███████   | 617/871 [00:27<00:11, 21.92it/s]













Epoch 2  Train data  Batch No : 617 Epoch Loss: 984.617 Loss : 1.579 Accuracy : 54.55% :  71%|███████   | 617/871 [00:27<00:11, 21.92it/s]













Epoch 2  Train data  Batch No : 618 E

...

Streaming output truncated to the last 5000 lines.
Epoch 16  Train data  Batch No : 613 Epoch Loss: 974.458 Loss : 1.583 Accuracy : 55.45% :  70%|███████   | 614/871 [00:28<00:12, 21.21it/s]













Epoch 16  Train data  Batch No : 614 Epoch Loss: 976.068 Loss : 1.609 Accuracy : 55.44% :  70%|███████   | 614/871 [00:28<00:12, 21.21it/s]













Epoch 16  Train data  Batch No : 615 Epoch Loss: 977.641 Loss : 1.573 Accuracy : 55.43% :  70%|███████   | 614/871 [00:28<00:12, 21.21it/s]













Epoch 16  Train data  Batch No : 616 Epoch Loss: 979.217 Loss : 1.577 Accuracy : 55.48% :  70%|███████   | 614/871 [00:28<00:12, 21.21it/s]













Epoch 16  Train data  Batch No : 616 Epoch Loss: 979.217 Loss : 1.577 Accuracy : 55.48% :  71%|███████   | 617/871 [00:28<00:11, 21.45it/s]













Epoch 16  Train data  Batch No : 617 Epoch Loss: 980.795 Loss : 1.578 Accuracy : 55.53% :  71%|███████   | 617/871 [00:28<00:11, 21.45it/s]













Epoch 16  Train data  Batch No 

printing iterator---> <torchtext.legacy.data.iterator.BucketIterator object at 0x7f0e63a19d90>


Streaming output truncated to the last 5000 lines.

Epoch 17  Train data  Batch No : 614 Epoch Loss: 975.904 Loss : 1.609 Accuracy : 54.98% :  70%|███████   | 613/871 [00:28<00:10, 24.96it/s]













Epoch 17  Train data  Batch No : 615 Epoch Loss: 977.513 Loss : 1.609 Accuracy : 54.98% :  70%|███████   | 613/871 [00:28<00:10, 24.96it/s]













Epoch 17  Train data  Batch No : 615 Epoch Loss: 977.513 Loss : 1.609 Accuracy : 54.98% :  71%|███████   | 616/871 [00:28<00:10, 23.43it/s]













Epoch 17  Train data  Batch No : 616 Epoch Loss: 979.091 Loss : 1.578 Accuracy : 55.03% :  71%|███████   | 616/871 [00:28<00:10, 23.43it/s]













Epoch 17  Train data  Batch No : 617 Epoch Loss: 980.700 Loss : 1.609 Accuracy : 55.02% :  71%|███████   | 616/871 [00:28<00:10, 23.43it/s]













Epoch 17  Train data  Batch No : 618 Epoch Loss: 982.310 Loss : 1.609 Accuracy : 55.00% :  71%|███████   | 616/871 [00:28<00:10, 23.43it/s]













Epoch 17  Train data  Batch No

printing iterator---> <torchtext.legacy.data.iterator.BucketIterator object at 0x7f0e63a19d90>


Epoch 17  TEST DATA Batch No : 13 Epoch Loss: 22.342  Loss : 1.580 | Accuracy : 8.12%:   0%|          | 0/35 [00:00<?, ?it/s]













Epoch 17  TEST DATA Batch No : 14 Epoch Loss: 23.949  Loss : 1.607 | Accuracy : 8.48%:   0%|          | 0/35 [00:00<?, ?it/s]













Epoch 17  TEST DATA Batch No : 15 Epoch Loss: 25.556  Loss : 1.607 | Accuracy : 8.93%:   0%|          | 0/35 [00:00<?, ?it/s]













Epoch 17  TEST DATA Batch No : 16 Epoch Loss: 27.148  Loss : 1.592 | Accuracy : 9.55%:   0%|          | 0/35 [00:00<?, ?it/s]













Epoch 17  TEST DATA Batch No : 17 Epoch Loss: 28.730  Loss : 1.582 | Accuracy : 10.00%:   0%|          | 0/35 [00:00<?, ?it/s]













Epoch 17  TEST DATA Batch No : 18 Epoch Loss: 30.334  Loss : 1.604 | Accuracy : 10.18%:   0%|          | 0/35 [00:00<?, ?it/s]













Epoch 17  TEST DATA Batch No : 19 Epoch Loss: 31.914  Loss : 1.580 | Accuracy : 10.45%:   0%|          | 0/35 [00:00<?, ?it/s]













Epoch 17  TEST DATA Batc